
# Preprocessing for MIT data with MinMax Scaler and Oversamling
The default preprocessing step is what we conclude in our report. 
You can check below what should be input for this notebook and what would be the output if necessary.

Processing **steps** for MIT dataset:   
    resample: Oversampling  
    rescaling: MinMaxScaler  


**Input** : Cleaned data with dummy target variable.   
mitbih_test_clean.csv   
mitbih_train_clean.csv

**Output** : Sampled and Scaled data with dummy target:   
mitbih_train_clean_minmax_oversampling.csv  
mitbih_test_clean_minmax_oversampling.csv


In [21]:
import sys
import os
data_path = ''
data_output_path = ''
# Check if the environment is Google Colab
if 'google.colab' in sys.modules:
    print("Running on Google Colab")
    # Install required libraries
    !pip install scikit-learn -q
    !pip install pandas -q
    !pip install numpy -q
    !pip install imbalanced-learn -q


    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    # set the path where the csv file stored in your google drive. 
    data_path = '/content/drive/MyDrive/Heartbeat_Project/'
    data_output_path = data_path
    
else:
    print("Running on local environment")

    current_path = os.getcwd()
    print("Current working directory:", current_path)
    data_path = '../data/raw/'
    data_output_path = '../data/processed/'


Running on local environment
Current working directory: g:\Meine Ablage\heartbeat-analysis-ai\notebooks


In [22]:
# Verify installation and import libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import RandomOverSampler


RawFiles = dict({
    'test': data_path +  'mitbih_test_clean.csv', # MIT data set in which we have dummy-coded the target variable
    'train': data_path +  'mitbih_train_clean.csv', # MIT data set in which we have dummy-coded the target variable
})

OutputFiles = dict({
    'test': data_output_path + 'mitbih_test_clean_minmax_oversamling.csv',
    'train': data_output_path + 'mitbih_train_clean_minmax_oversampling.csv',
})

In [23]:
def addColumnsToDataframe(df):
    """
    As the dataset is composed with 188 columns with the 188th columns as the category values,
    so we give the last column the name 'target', others named with 'c_182'
    """
    num_columns= df.shape[1]
    feature_col_name = ['c_' + str(i) for i in range(0, num_columns - 1)]
    df_columns = feature_col_name + ['target']
    df.columns = df_columns
    return df

# def convertColumnAsInt(df, column):
#     """
#     As the category value is in float type. We want to get the int to identify the category.
#     """
#     df[column] = df[column].astype(int)
#     return df


In [24]:
def convertColumnAsInt(df, column):
    df[column] = pd.to_numeric(df[column], errors='coerce') # convert to numeric to handle NaN values
    df.dropna(subset=[column], inplace=True)  # drop the rows with NaN values
    df[column] = df[column].astype(int)  # convert to int
    return df


In [25]:
mitbih_train = pd.read_csv(RawFiles.get('train'), header=None ) 
mitbih_test = pd.read_csv(RawFiles.get('test'), header=None )

mitbih_train = addColumnsToDataframe(mitbih_train)
mitbih_train = convertColumnAsInt(mitbih_train, 'target')

mitbih_test = addColumnsToDataframe(mitbih_test)
mitbih_test = convertColumnAsInt(mitbih_test, 'target')

# target value and meanings
class_mapping = {
    0: 'Normal',
    1: 'Abnormal'
}

#drop null value  
mitbih_train = mitbih_train.dropna(how='any')
mitbih_test = mitbih_test.dropna(how='any')




C:\Users\Isabell Gurstein\AppData\Local\Temp\ipykernel_42316\433990335.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187) have mixed types. Specify dtype option on import or set low_memory=False.
  mitbih_train = pd.read_csv(RawFiles.get('train'), header=None )
C:\Users\Isabell Gurstein\AppData\Local\Temp\ipykernel_42316\433990335.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8

In [26]:
mitbih_train["target"].value_counts()

target
0    72471
1     8652
Name: count, dtype: int64

In [27]:
mitbih_test["target"].value_counts()

target
0    18118
1     2166
Name: count, dtype: int64

In [28]:
# split train test set before resampling

X_train = mitbih_train.drop(columns=['target'], inplace=False) # drop the target column
X_test = mitbih_test.drop(columns=['target'], inplace=False) # drop the target column

y_train = mitbih_train['target'] # only the target column
y_test = mitbih_test['target'] # only the target column


# Resampling the train data with Oversampling before scaling with MinMaxScaler
ros = RandomOverSampler(random_state=0)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Scaling with MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)


In [29]:
# convert resampling rescaling data back to dataframe to concat
X_train_scaled_df =  pd.DataFrame(X_train_scaled, columns=[f'c_{i}' for i in range(X_train_scaled.shape[1])])
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=[f'c_{i}' for i in range(X_test_scaled.shape[1])])
y_train_resampled_df = pd.DataFrame(y_train_resampled, columns=['target'])
y_test_df = pd.DataFrame(y_test, columns=['target']) 


# concat X_train, y_train/ X_test, y_test
mitbih_train_clean_default = pd.concat(
    [
        X_train_scaled_df,
        y_train_resampled_df
    ], axis=1)

mitbih_test_clean_default = pd.concat(
    [  
        X_test_scaled_df,
        y_test_df.reset_index(drop=True)
    ], axis=1)


In [30]:
#save clean data to 
# mitbih_train_clean_default.csv
# mitbih_test_clean_default.csv

mitbih_train_clean_default.to_csv(OutputFiles.get('train'), index=False)
mitbih_test_clean_default.to_csv(OutputFiles.get('test'), index=False)